In [0]:
import random , os
import multiprocessing as mp
from queue import Empty
import math
import time

I took the speeding up python notebook: Speeding up Python and expanded it further.

Here is the link to the original notebook:
( https://colab.research.google.com/drive/1nMDtWcVZCT9q1VWen5rXL8ZHVlxn2KnL )

-Patrick Kong



#This is the presentation for code optimization for python. We will explore the tools we need in order to achieve this.

#Code timing

Noticing the run time on a code is a good start towards optimizing a code since
we can see how fast it will run.

So we'll be looking at these commands:

%time :  times the execution time of a single statement.

%timeit :times the loops of a single statement for more accuracy. 



In [0]:
#for this example we'll use a simple function to demonstrate the %time command

%time sigma = sum( range( 100 ) )

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 13.1 µs


For a simple statement we can use %time like this. If we need to use it over a loop iterations
we will need to change the way it will be called.

In [0]:
#let's try doing a looped statement of the same thing

%%time

sigma = 0
for i in range( 100 ):
  sigma += + i



CPU times: user 23 µs, sys: 4 µs, total: 27 µs
Wall time: 31.7 µs


Next we'll be using the %timeit function. We could also use the same functions to demonstrate
the difference between the two commands.

In [0]:
%timeit sigma = sum( range( 100 ) )

1000000 loops, best of 3: 1.3 µs per loop


So %timeit tests the statement by running through loops and gives us the best of 3 timings for the loops

In [0]:
%%timeit

sigma = 0
for i in range( 100 ):
  sigma += + i


100000 loops, best of 3: 7.4 µs per loop


Notice how %timeit lowered the number of loops. 

For slower codes, %timeit will lower the number of
loops it will test.

#Decorators

Before we start diving into this. Let's start talking about decorators.

Decorators are functions that takes in another function and extends the behavior of the function.

This is great for programming even if you aren't using it for optimizations.

Here, we will look at a simple function:

In [0]:
#coding example

def decoratorExample( func ):
  def wrapperFunc():
    print( "this is before the function in the wrapper goes off." )
    func()
    print( "the function in the wrapper has activated." )
    
  return wrapperFunc
  
def beep():
  for x in range( 10 ):
    print( x )
    
#two ways to call on a decorator

#first way:
beep = decoratorExample( beep )
beep()


this is before the function in the wrapper goes off.
0
1
2
3
4
5
6
7
8
9
the function in the wrapper has activated.


In [0]:
#coding example

def decoratorFunc( func ):
  def wrapperFunc( *args, **kwargs ):
    print( "this is before the function in the wrapper goes off." )
    func( )
    print( "the function in the wrapper has activated." )
    
  return wrapperFunc

#second way is to use syntactic sugar:

@decoratorExample
def beep():
  for x in range( 10 ):
    print( x )
    
#doing it this way allows us to just call the function itself

beep()

this is before the function in the wrapper goes off.
0
1
2
3
4
5
6
7
8
9
the function in the wrapper has activated.


#We could also use decorator to make a register or plugins for functions.



In [0]:
#global dictionary for registers
PLUGINS = dict()

def registerFunc( func ):
  
  

We've learned about timing function. We could literally use this to make a custom timing!

In [0]:
import time

def custom_timing( func ):
  
  def timing_func( *args, **kwargs ):
    
    #start timing
    start = time.perf_counter()
    result = func( *args, **kwargs )
    end = time.perf_counter()
    
    timing = end - start
    print( 'time: %f ' % ( timing ) )
    
    return result
  
  return timing_func

@custom_timing
def fibonacci( n ):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci( n - 1 ) + fibonacci( n - 2 )

#you'll notice that in order to make this work we'll have to use a different timing function
#because recursion will cause the decorator to wrap each instance of fibonacci!


fibonacci( 2 )

time: 0.000000 
time: 0.000001 
time: 0.000242 


1

#Code Profiling

We understand how to time our codes now. 

Timing the code gives us more information and is fine for understanding the run times.

However, we can take this a step further. This is the next step after we've timed our code.

Here is where we will identify performance bottlenecks that exist in codes.

To accomplish this, we will look at **cProfiler** and **line_profiler**

#cProfiler.

This is a built-in profiler and should be the first alternative after you've gained some intuition timing the functions.

The reason is because this will be a little slower than using the timing functions. You'll use this if you need to gain more insight about the run times for your code.

We'll use a bubble sort for code profiling.

A decorator function is a function that takes in a function as an input and returns a function for output.

we'll be using a decorator function to control what our cprofiler will do.

for more information on commands we'll use this documentation:

Python 3: https://docs.python.org/3/library/profile.html



In [0]:
import cProfile
import numpy as np

An import will be needed to make this work.

In [0]:
#start implementing cProfiler here!

def cprofiler( function ):
  
  #the profiling function
  def profile_func( *args, **kwargs ):
    
    #Using it this way allows the profile to display results without
    #writing the profile data to a file
    profile = cProfile.Profile()
    try:
      #starts the collecting of profile data
      profile.enable()
      
      results = function( *args, **kwargs )
      #stops collecting of profile data
      profile.disable()
      
      return results
    finally:
      #we use finally as a finish procedure
      #in this case print_stats will print results to standard out.
      #for the sake of the demo we'll comment this out
      profile.print_stats()
  return profile_func

@cprofiler
def bubble_sort( list ):
    # TODO your code here
    
    
    is_sorted = False
    num_sort = 0
    temp = 0
    while is_sorted == False:
      
        #this variable is reset at the start of every loop
        num_sort = 0
        for index in range( len(list) - 1 ):
          
            #if the preceding number in the array is smaller
            #a swap will be done.
            if list[ index ] > list[ index + 1 ]:
                temp = list[ index ]
                list[ index ] = list[ index + 1 ]
                list[ index + 1 ] = temp
                num_sort += 1
                
        #if nothing is sorted then the flag is set true
        #this will prevent another loop from starting
        if num_sort == 0:
                is_sorted = True
            
    #the newly sorted list is returned
    return list

@cprofiler
def create_list():
  list = np.zeros( 1000 )
  x = 0

  for i in range( 1000, 0, -1 ):
    list[ x ] = i
    x += 1
  return list

if __name__ == '__main__':
  #we'll comment out the output for the sake of the demo
  list = create_list()
  #print( "before: ", list )
  bubble_sort( list )
  #print( "after: ", list )



NameError: ignored

We can see the timing for both the create_list and bubble_sort method with this cProfile.

**cProfiler**


pros: 
- no external dependencies
- fast
- useful for quick checks.

cons:
- limited information which usually doesn't help if you need deeper debugging.
- reports aren't easily understood.

#line_profiler

Let's take a look at line profiler next. We'll use the same code for this one just to show what it can do.

For documentation:

visit: https://github.com/rkern/line_profiler#line-profiler

In [0]:
!pip install line_profiler
from line_profiler import LineProfiler

     |████████████████████████████████| 92kB 5.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/05/7d/9b/aafbe8d78dc2b2c644d2efd2f060ab3258143860142575193a
Successfully built line-profiler


As you've noticed line profiler isn't built in. However, I think it's still a very good profiler to use if you need line by line analysis for the code.



In [0]:
#implement line profiler function here!

def profiler_function( function ):
  def profile_function( *args, **kwargs ):
    try:
      profiler = LineProfiler()
      
      #this add_function informs 
      #the line profiler which function to profile
      profiler.add_function( function )
      
      #enable_by_count is the same as enable for cProfiler
      #except for line profiler this is safe to used on nested functions
      profiler.enable_by_count()
      
      #please note if you want to disable then you'll use:
      #disable_by_count()
      
      return function( *args, **kwargs )
    finally:
      profiler.print_stats()
  return profile_function

'''
except ImportError:
  def line_profiler():
    def inner( function ):
      def nothing( *args, **kwargs )
        return function( *args, **kwargs )
      return nothing
    return inner
'''

@profiler_function
def bubble_sort(list):
    # TODO your code here
    is_sorted = False
    num_sort = 0
    temp = 0
    while is_sorted == False:
        num_sort = 0
        for index in range(len(list)-1):
            if list[index] > list[index +1]:
                temp = list[index]
                list[index] = list[index+1]
                list[index+1] = temp
                num_sort += 1
        if num_sort == 0:
                is_sorted = True
    return list

@profiler_function
def create_list():
  list = np.zeros( 1000 )
  x = 0

  for i in range( 1000, 0, -1 ):
    list[ x ] = i
    x += 1
  return list

if __name__ == '__main__':
  #we'll comment out the output for the sake of the demo
  list = create_list()
  #print( "before: ", list )
  bubble_sort(list)
  #print( "after: ", list )


Timer unit: 1e-06 s

Total time: 0.002555 s
File: <ipython-input-9-29a6d464fd58>
Function: create_list at line 43

Line #      Hits         Time  Per Hit   % Time  Line Contents
    43                                           @line_profiler
    44                                           def create_list():
    45         1         34.0     34.0      1.3    list = np.zeros( 1000 )
    46         1          2.0      2.0      0.1    x = 0
    47                                           
    48      1001        569.0      0.6     22.3    for i in range( 1000, 0, -1 ):
    49      1000       1264.0      1.3     49.5      list[ x ] = i
    50      1000        685.0      0.7     26.8      x += 1
    51         1          1.0      1.0      0.0    return list

Timer unit: 1e-06 s

Total time: 1.91243 s
File: <ipython-input-9-29a6d464fd58>
Function: bubble_sort at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           @line_pro


We can see the number of operations, run time and miscellaneous details in a line by line fashion.
This is a great tool for checking on performance bottlenecks, if you still need to gather more information about your code.

**Line Profiler**


Pros:
- intiuitive
- reports are detailed
- can follow functions in third party libraries

Cons:
- huge overhead
- because it is slow, this shouldn't be used for benchmarking
- it isn't built in so, you'll have to import externally
- timing issues arise with nested functions
- not stackable

#parallelization.

In [0]:
#We use cpu_count method to give us the max number of
#CPUs available

print( "number of CPUs we can use: ", mp.cpu_count() )

number of CPUs we can use:  2


We can run a simple code to see what happens when we run parallel processes

With this there are two things we need to consider for parallelization:

**Syncronuous** and **Asyncronous**

A **synchronous** operation **blocks** a process till the operation completes.

An **asynchronous** operation is **non-blocking** and only initiates the operation.


There are methods we can consider with this:

**Synchronous:**

**apply( func, args, kwds )**,
**map( func, iterable, chunksize )**,
**starmap( func, iterable, chunksize )**

**Asynchronous:**

**apply_async( func, args, kwds, callback )**,
**map_async( func, iterable, chunksize, callback )**,
**starmap_async( func, iterable, chuncksize, callback )**

In [0]:
def functionCount( count ):
  time.sleep( 2 )
  return count*count

results_list = []
def result_log( count ):
  results_list.append( count )

def pool_demo():
  #this will give us 4 processes to work with
  pool = mp.Pool( )
  for i in range( 10 ):
    #apply works by blocking until the result is ready
    result_log( pool.apply( functionCount, args = ( i, ) ) )
    
  #this prevents any more tasks from being submitted into the pool
  pool.close()

  #this will wait for worker processes to exit
  pool.join()
  
  print( "outputting results from apply method:\n" )
  print( results_list )

if __name__ == '__main__':
    ti = time.time()
    pool_demo()
    print( "Time it took to run this function: ", time.time() - ti )

outputting results from apply method:

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Time it took to run this function:  20.088012218475342


In [0]:
def functionCount( count ):
  time.sleep( 2 )
  return count*count

apply_list = []
results_list = []
def result_log( count ):
  results_list.append( count )

def pool_demo():
  #this will give us 4 processes to work with
  pool = mp.Pool( 4 )
  for i in range( 10 ):
    #apply_sync works by processing and returning the results when it's ready
    pool.apply_async( functionCount, args = ( i, ), callback = result_log )

  #this prevents any more tasks from being submitted into the pool
  pool.close()

  #this will wait for worker processes to exit
  pool.join()
  
  print( "outputting results from apply_async method:\n")
  print( results_list )
  
if __name__ == '__main__':
    ti = time.time()
    pool_demo()
    print( "Time it took to run this function: ", time.time() - ti )

outputting results from apply_async method:

[0, 1, 4, 9, 25, 16, 36, 49, 64, 81]
Time it took to run this function:  6.059282064437866


We can see that the numbers are initiated in order with **apply** method. 

However, this might not always be the case as **apply_async** method doesn't
always output in the order that corresponds to the iterations.

Note that with **apply_async** be using callback.

This is why we have **map** and **apply** methods available if we need the 
processes to be blocked until the result is ready.

In [0]:
!pip install cython
%load_ext Cython

from distutils.core import setup
from Cython.Build import cythonize

Installing cython and loading the module

#Cython

The next thing to consider would be to use another language to run the python code.

C + Python = Cython

Cython addresses the shortcomings that exists in python:


**execution speed, GIL-free concurrency, absence of type checking and not creating an executable.**


3 things we'll be looking at:

**def** - regular python function, calls from Python only.


**cdef** - cython only functions, can't access these from python-only code, must access within Cython, since there will be no C translation to Python for these.


**cpdef** - C and Python. Will create a C function and a wrapper for Python. Why not *always* use cpdef? In some cases, you might have C only pointer, like a C array.

#def

Let's use a sieve function since it will show us the speed between these 3 declarations.

In [0]:
def sieve( size ):
  upper_limit = int( math.sqrt( size ) )
  
  #initializing an array of zeroes for both prime and non prime arrays
  #numpy will give us the means to create an array of appropriate size
  non_prime = np.zeros( ( size ) )
  prime = np.zeros( ( size ) )
  
  tally = 0
  
  for m in range( 2, size ):
    
    '''
    this will take the numbers to compare against non primes to
    create a list of prime numbers
    '''
    
    if( m != non_prime[ m ] ):
      prime[ tally ] = m
      #for demo purposes we'll comment out this print statement
      #print( '{}' .format( int( prime[ tally ] ) ) )
      tally += 1
      i = 0
    
    #this while statement will fill up the non prime numbers
    while( ( i * m ) < size ):
      if( i * m != non_prime[ i * m ] ):
        non_prime[ i * m ] = i * m
      i += 1
      
def sieve_demo( size ):
  primes = [True for _ in range( size + 1 ) ]
  primes[0] = primes[1] = False
  upper_limit = int(math.sqrt( size ))
  for i in range(2, upper_limit+1):
      if not primes[i]:
          continue
      for j in range(2*i, size+1, i):
          primes[j] = False
  return [x for x in range(2, size+1) if primes[x]]
  
if __name__ == '__main__':
    #ti = time.time()
    %timeit sieve( 200000 )
    #print( "Time it took to run this function: ", time.time() - ti )
    %timeit sieve_demo( 200000 )

1 loop, best of 3: 790 ms per loop
1 loop, best of 3: 261 ms per loop


The timing is still pretty fast, all things considering. Now let's speed it up further.

We'll take this same function and Cythonize it.

For this we will use **cdef**. This will contain a lot of gotchas considering that
we will have to make this code operate like **C code**.

#cdef

because we'll be using c for this cythonized code, we'll also be using **malloc** and **free** for the arrays.

**malloc** will give us the memory allocation for the arrays.


**free** will free up the memory taken up array when we are done.

In [0]:
#this will be the cythonized version of sieve

%%cython
from libc.math cimport sqrt
from libc.stdlib cimport malloc, free
from libc.time cimport time,time_t


def sieve_c( size ):
  upper_limit = int( sqrt( size ) )
  
  #we initialize non prime and prime as a pointer
  #malloc will allow us to dynamically allocate memory
  #we have 2 arrays with size+1 of 4 bytes
  cdef int *non_prime = < int * > malloc( ( size + 1 ) * sizeof( int ) )
  cdef int *prime = < int * > malloc( ( size + 1 ) * sizeof( int ) )
  
  cdef int tally = 0, i = 0
  
  for m in range( 2, size ):
    
    '''
    this will take the numbers to compare against non primes to
    create a list of prime numbers
    '''
    
    if( m != non_prime[ m ] ):
      prime[ tally ] = m
      #for demo purposes we'll comment out this print statement
      #print( '{}' .format( int( prime[ tally ] ) ) )
      tally += 1
      i = 0
    
    #this while statement will fill up the non prime numbers
    while( ( i * m ) < size ):
      if( i * m != non_prime[ i * m ] ):
        non_prime[ i * m ] = i * m
      i += 1
  free( non_prime )
  free( prime )
  
#this is the better version of sieve
def sieve_demo_c(up_to):  
    cdef int i, j
    cdef bint *primes = <bint *>malloc((up_to+1) * sizeof(bint))
    for i in range(up_to+1):
        primes[i] = 1
    primes[0] = primes[1] = False
    cdef int upper_limit = int(sqrt(up_to))
    for i in range(2, upper_limit+1):
        if not primes[i]:
            continue
        j = 2*i
        while j < up_to + 1:
            primes[j] = False
            j += i
    free( primes )


In [0]:
#this cythonized function works!
#ti = time.time()
%timeit sieve_c( 200000 )
#print( "Time it took to run this function: ", time.time() - ti )

#ti = time.time()
%timeit sieve_demo_c( 200000 )
#print( "Time it took to run this function: ", time.time() - ti )

10 loops, best of 3: 37.3 ms per loop
100 loops, best of 3: 20.2 ms per loop


The speed is drastically improved from the original sieve function.

**cpdef** version would not be too different since we would have the best of both worlds with python
and C.

This means that if we Cython code, the runtime will optimize towards C efficiency and if we use the code with python code, the runtime will be like that of python.

While we get the flexibility of both python and cython with **cpdef**.

#caching

Caching is a really great way to optimize expensive code. We'll be using a technique called memoization (or memoisation) to implement caching.

To demonstrate this we'll be using a **Fibonacci function** since this function is really **expensive (slow and resource intensive)**.

In [0]:
#implement memoization function here!

def memoization( function ):
  cache = dict()
  
  def memoized_function( *args ):
    #idea here is to take the argument and see if it exist in our cache
    #if the it does, we'll return the cache value.
    if args in cache:
      return cache[ args ]
    
    
    result = function( *args )
    cache[ args ] = result
    
    return result
  
  return memoized_function


#fibonacci function
@memoization
def fibonacci( n ):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci( n - 1 ) + fibonacci( n - 2 )

if __name__ == '__main__':
  #for this example we could just use %timeit to demonstrate runtime
  %timeit fibonacci( 10 )

The slowest run took 45.17 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 762 ns per loop


Here we have the fibonacci function. We'll run it first to see how it performs.

Next we'll try using our decorator function: memoization and see how it performs.

**BTW python has higher order functions built in which features this function:**

**lru_cache( maxsize = 128, typed = False )**

the **lru** stands for **least recently used**

In this implementation of cache, a cache is made with each item containing age bits. This indicate how recent the item is.

NOTE: you'll notice that **typed** is set to **False**. If it was set to **True** it means that each items of different types will be cached separately.

This means that given **f( 3 )** and **f( 3.0 )**, each will be handled as different calls with different results.

Let's take the same function and try it out!

In [0]:
from functools import lru_cache

@lru_cache( maxsize = 32 )
def fibonacci( n ):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci( n - 1 ) + fibonacci( n - 2 )

if __name__ == '__main__':
  #for this example we could just use %timeit to demonstrate runtime
  %timeit fibonacci( 10 )

The slowest run took 45.41 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 337 ns per loop


An even faster result! 

Caching is a great tool that can be used to optimize our code. 

We generally want to use this on expensive codes.

for documentations:

https://docs.python.org/3.4/library/functools.html

#Numba

Numba is a JIT compiler for python.

This is what the decorator looks like:

@numba.**jit**( **signature = None,
                        nopython = False,
                        nogil = False,
                        cache = False,
                        forceobj = False,
                        parallel = False,
                        error_model = 'python',
                        fastmath = False,
                        locals = {}** )
                        
For jit tho we will only be looking at **signatures, nopython, cache.**

There's no point in using **forceobj** since it just means we are forcing it into object mode ( which we don't want btw )

**nogil** means we are taking off the global interpreter lock, that's a generally a bucket of worms we do not want to open.

**error_model** controls the divide by zero behavior.

**fast_math** allows the use of unsafe floating point transforms.



In [0]:
!pip install numba
from numba import jit, njit

We'll need to install numba in order to use it!

Now for numba you'll see that there is both @jit and @njit.

@jit: nopython mode
         / nobject mode
         
@njit: nopython mode

There is a slight difference between the two:

@njit is shortened form of: @jit( nopython = True )

This nopython mode locks the interpreter from ever trying to handle it as a python object.

It means that if a code requires the interpreter to handle it as an python object ( object mode ) @njit will return an error, whereas, if you use just @jit the interpreter will use object mode as necessary. Something to keep in mind.

For this example let's take a previous code we know has a lot of loops and see if there's an improvement.

In [0]:
from functools import wraps


@njit
def sieve_default( size ):
  upper_limit = int( math.sqrt( size ) )
  
  #initializing an array of zeroes for both prime and non prime arrays
  #numpy will give us the means to create an array of appropriate size
  non_prime = np.zeros( ( size ) )
  prime = np.zeros( ( size ) )
  
  tally = 0
  
  for m in range( 2, size ):
    
    '''
    this will take the numbers to compare against non primes to
    create a list of prime numbers
    '''
    
    if( m != non_prime[ m ] ):
      prime[ tally ] = m
      #for demo purposes we'll comment out this print statement
      #print( '{}' .format( int( prime[ tally ] ) ) )
      tally += 1
      i = 0
    
    #this while statement will fill up the non prime numbers
    while( ( i * m ) < size ):
      if( i * m != non_prime[ i * m ] ):
        non_prime[ i * m ] = i * m
      i += 1
      
if __name__ == '__main__':
  #we'll comment out the output for the sake of the demo
  
  %timeit sieve_default( 100 )
  %timeit sieve_cache( 100 )

SyntaxError: ignored

#using signatures with @jit

There's two types of compilations:

Lazy compilation:

This let's Numba decide how to optimize.

Eager compilation:

This tells Numba what data types to expect.


Using signature isn't really a requirement, however, it is a great way to maintain control over your function should the need arise.

In [0]:
#coding example here



@jit( int32( int32, int32 ) )
def simple_func( x, y ):
  return x + y

#we could also use @njit( int32( int32, int32 ) ) or
#@jit( [ int32( int32, int32 ) ], nopython = True )

#the idea for this jit is that it will take the two parameters of int
#after the object is returned an int

#If we simply have: ( int, int ) we would have eager compilation for the function.
#However, we are leaving it up to jit to infer the return type.

print( simple_func( 3, 7 ) )

#returns 10

#Parallelization with Numba

You can do parallel processing with Numba simply by doing this:

**@njit( parallel = True )**

We can also create diagnostics by calling:

**parallel_diagnostics( level )**

This diagnostic is a report on the optimizations being done during the parallelization process.


**level** controls the verbosity of the diagnostic report. **1 being the least, whereas 4 being the most.**

In [0]:
#coding example here

@njit( parallel = True )
def square_num( x ):
  square_inc = 0
  for i in range( x ):
    square_inc = square_inc + i ** 2
  
  return square_inc
    
print( square_num( 100 ) )

#going to generate a very verbose report
square_num( 10 ).parallel_diagnostics( level = 4 )

328350


#Caching with Numba

You can do caching with Numba simply by doing this:

@njit( cache = True )

In [0]:
#coding example here

@jit( cache = True )
def fibonacci( n ):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci( n - 1 ) + fibonacci( n - 2 )

if __name__ == '__main__':
  #for this example we could just use %timeit to demonstrate runtime
  %timeit fibonacci( 1000 )

#using @cfunc with Numba

@numba.**cfunc**( **signature, nopython = False, cache = False, locals = {}** )

locals is a mapping of local variable names to Types and signatures.

**note: c callbacks does not support object mode. This means we are running like it's @njit.**


You can use numba to compile a function callable from c code.

This will require you to use signatures.

Signatures are data types that are specified for the decorator parameters.

This example we will use cfunc and will use .ctypes to callback the object.

In [0]:
from numba import cfunc
#coding example here

@cfunc( "float64(float64, float64)" )
def add( x, y ):
  return x + y

#we wrapped the add function using cfunc
#and the outside will cast the type as float64 at the end

#using ctypes to get callback from the function add.
print( add.ctypes( 4.0, 5.0 ) )

#we could also use cfunc to deal with pointers and arrays



In [0]:
from numba import types, carray

c_sig = types.void( types.CPointer( types.double ),
                    types.CPointer( types.double ),
                    types.intc, types.intc )

@cfunc( c_sig )
def array_example( arr1, arr2, x, y ):
  array1 = carray( arr1, ( x, y ) )
  array2 = carray( arr2, ( x, y ) )
  
  for i in range( x ):
    for j in range( y ):
      array2[ i, j ] = 2 * array1[ i , j ]